In [1]:
%pip install pandas
%pip install plotly
%pip install opencv-python
%pip install nbformat

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import cv2
import os
import numpy as np
import plotly.express as px

In [3]:
fer2013_df = pd.read_csv("../../data/raw/fer2013/fer2013.csv", index_col=False)
fer2013_df["image_name"] = ""
fer2013_df.rename(columns={"emotion" : "expression_label"}, inplace=True)
fer2013_df.drop(columns="Usage", inplace=True)

fer2013_df.head()

,expression_label,pixels,image_name
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,


In [4]:
expression_labels = {
    0: "Angry",
    1: "Disgust",
    2: "Fear",
    3: "Happy",
    4: "Sad",
    5: "Surprise",
    6: "Neutral",
}

In [5]:
for index, row in fer2013_df.iterrows():
    expression = expression_labels.get(row["expression_label"]).lower()
    fer2013_df.loc[index, "expression_label"] = expression
    fer2013_df.loc[index, "image_name"] = str("fer2013_"+ expression +"_" + str(index) + ".jpg")


fer2013_df["expression_label"] = fer2013_df["expression_label"].astype("category")

fer2013_df.head()

C:\Users\pedro\AppData\Local\Temp\ipykernel_25292\71553197.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'angry' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  fer2013_df.loc[index, "expression_label"] = expression


,expression_label,pixels,image_name
0,angry,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,fer2013_angry_0.jpg
1,angry,151 150 147 155 148 133 111 140 170 174 182 15...,fer2013_angry_1.jpg
2,fear,231 212 156 164 174 138 161 173 182 200 106 38...,fer2013_fear_2.jpg
3,sad,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,fer2013_sad_3.jpg
4,neutral,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,fer2013_neutral_4.jpg


In [6]:
processed_folder_path = "../../data/processed/fer2013/cropped/"
if not os.path.exists(processed_folder_path):
    os.makedirs(processed_folder_path)
for index, row in fer2013_df.iterrows():
    image_string = row["pixels"].split(" ")
    image_data = np.asarray(image_string, dtype=np.uint8).reshape(48, 48)
    cv2.imwrite(os.path.join(processed_folder_path, row["image_name"]),image_data)
    

# Combine FER2013 with EXPW

In [7]:
expw_label_path = "../../data/processed/expw/new_label.csv"

expw_label_df = pd.read_csv(expw_label_path, index_col=False)

expw_label_df.head()

,image_name,expression_label
0,angry_0.jpg,angry
1,angry_1.jpg,angry
2,fear_2.jpg,fear
3,happy_3.jpg,happy
4,angry_4.jpg,angry


In [8]:
fer2013_label_df = fer2013_df.drop(columns="pixels")

fer2013_label_df.head()

,expression_label,image_name
0,angry,fer2013_angry_0.jpg
1,angry,fer2013_angry_1.jpg
2,fear,fer2013_fear_2.jpg
3,sad,fer2013_sad_3.jpg
4,neutral,fer2013_neutral_4.jpg


In [9]:
combined_df = pd.concat([expw_label_df, fer2013_label_df], ignore_index=True)

combined_df.head()

,image_name,expression_label
0,angry_0.jpg,angry
1,angry_1.jpg,angry
2,fear_2.jpg,fear
3,happy_3.jpg,happy
4,angry_4.jpg,angry


In [10]:
len(combined_df)

127680

In [11]:
grouped_count = combined_df.groupby(["expression_label"])[
    "expression_label"
].count()

grouped_count

expression_label
angry        8624
disgust      4542
fear         6209
happy       39526
neutral     41081
sad         16636
surprise    11062
Name: expression_label, dtype: int64

In [12]:
fig = px.bar(
    x=grouped_count.index,
    y=grouped_count.values,
    color=grouped_count.index,
)

fig.update_layout(
    title="Total images for each label (expw & fer2013 combined)",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

In [13]:
happy_rows_to_drop = combined_df[combined_df["expression_label"] == "happy"].sample(frac=0.7, random_state=42)
neutral_rows_to_drop = combined_df[combined_df["expression_label"] == "neutral"].sample(frac=0.7, random_state=42)
sad_rows_to_drop = combined_df[combined_df["expression_label"] == "sad"].sample(frac=0.25, random_state=42)

In [14]:
undersampled_df = combined_df.drop(happy_rows_to_drop.index)
undersampled_df = undersampled_df.drop(neutral_rows_to_drop.index)
undersampled_df = undersampled_df.drop(sad_rows_to_drop.index)
undersampled_df

,image_name,expression_label
0,angry_0.jpg,angry
1,angry_1.jpg,angry
2,fear_2.jpg,fear
4,angry_4.jpg,angry
5,angry_5.jpg,angry
...,...,...
127672,fer2013_fear_35879.jpg,fear
127673,fer2013_fear_35880.jpg,fear
127674,fer2013_angry_35881.jpg,angry
127677,fer2013_angry_35884.jpg,angry


In [15]:
grouped_undersampled_count = undersampled_df.groupby(["expression_label"])[
    "expression_label"
].count()

grouped_undersampled_count

expression_label
angry        8624
disgust      4542
fear         6209
happy       11858
neutral     12324
sad         12477
surprise    11062
Name: expression_label, dtype: int64

In [16]:
fig = px.bar(
    x=grouped_undersampled_count.index,
    y=grouped_undersampled_count.values,
    color=grouped_undersampled_count.index,
)

fig.update_layout(
    title="Total images for each label (expw & fer2013 undersampled)",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

In [17]:
label_path = "../../data/processed/combined/combined_label.csv"
combined_img_folder_path = "../../data/processed/combined/img/"

if not os.path.exists(combined_img_folder_path):
    os.makedirs(combined_img_folder_path)

undersampled_df.to_csv(label_path, index=False)

In [18]:
expw_images_folder_path = "../../data/processed/expw/cropped/"
fer2013_images_folder_path = "../../data/processed/fer2013/cropped/"

In [19]:
import shutil


for index, row in undersampled_df.iterrows():
    dst = combined_img_folder_path + row["image_name"]
    if "fer2013" in row["image_name"]:
        src = fer2013_images_folder_path + row["image_name"]
    else:
        src = expw_images_folder_path + row["image_name"]
    
    shutil.copy(src, dst)